**Курс** Специализация Data Science

**Дипломный проект.** Бриф учебного кейса 

«Модель прогнозирования стоимости жилья для агентства недвижимости»

**Цель**: разработать модель, которая позволила бы агентству недвижимости обойти конкурентов по скорости и качеству совершения сделок.

**Описание данных**:
- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать)

In [21]:
import numpy as np 
import pandas as pd

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import os

# для графиков
import seaborn as sns
import plotly.subplots as spimport
import matplotlib.pyplot as plt

# для нормализации
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from catboost import CatBoostRegressor
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
%matplotlib inline

In [22]:
# фиусируем RANDOM_SEED
RANDOM_SEED = 42

In [23]:
# фиксируем версию пакетов
!pip freeze > requirements.txt

In [7]:
df = pd.read_csv('data/cleaned_data_model.csv')
display(df.head())
df.info()

,baths,sqft,zipcode,target,private_pool_final,school_rating _mean,school_dist_min,Heating_final,Cooling_encoded,Parking_encoded,...,propertyType_mobile home,propertyType_multi family,propertyType_other,propertyType_ranch,propertyType_single family,propertyType_townhouse,propertyType_unknown,state_label,year_label,city_label
0,-2.268674,-4.111717,28387,-0.584591,False,-0.478034,-2.200214,True,False,False,...,0,0,0,0,1,0,0,18,202,1287
1,-2.674134,-4.510277,99216,-0.883500,False,-0.693145,-2.810405,False,False,False,...,0,0,0,0,1,0,0,32,202,1295
2,-3.367267,-3.898779,50401,-1.119223,False,-0.733967,-1.621481,True,True,False,...,0,0,0,0,1,0,0,7,153,815
3,-2.674134,-4.519051,77080,-0.877086,False,-0.916288,-3.038531,True,True,True,...,0,0,0,0,1,0,0,28,202,617
4,-3.367267,-4.914413,11354,-0.114288,False,-0.967581,-3.246166,False,False,True,...,0,0,0,0,0,0,0,21,148,454


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197252 entries, 0 to 197251
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   baths                       197252 non-null  float64
 1   sqft                        197252 non-null  float64
 2   zipcode                     197252 non-null  int64  
 3   target                      197252 non-null  float64
 4   private_pool_final          197252 non-null  bool   
 5   school_rating _mean         197252 non-null  float64
 6   school_dist_min             197252 non-null  float64
 7   Heating_final               197252 non-null  bool   
 8   Cooling_encoded             197252 non-null  bool   
 9   Parking_encoded             197252 non-null  bool   
 10  status_Active               197252 non-null  int64  
 11  status_Auction              197252 non-null  int64  
 12  status_Coming Soon          197252 non-null  int64  
 13  status_Conting

Разделим данные в датасете.

In [8]:
y = df['target']
X = df.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

 **LinearRegression**

In [9]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# вычисляем коэффициенты регрессии
model.fit(X_train, y_train)

# делаем предсказания с помощью модели
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# вычисляем требуемые метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred)
mse_test = metrics.mean_squared_error(y_test, y_test_pred)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.31
Test MSE: 0.29
Train MAE: 0.39
Test MAE: 0.39
Train R2: 0.30
Test R2: 0.31


Обратим внимание, что результаты значений на обеих выборках или совпадают, или не сильно отличаются,
что свидетельствует о том, что модель не переобучилась.
- Среднеквадратическая ошибка (Mean Squared Error, MSE) для обучающей выборки составила 0.31, а для тестовой выборки  0.29. Низкие значения MSE указывают на более точные предсказания модели.
- Средняя абсолютная ошибка (Mean Absolute Error, MAE) в обеих выборках составляет 0.39. Низкие значения MAE также говорят о хорошей точности предсказания, но 0.39 не такой уж низкий показатель.
- Коэффициент детерминации (R2) составляет 0.30 как для обучающей, и 0.31 для тестовой выборки, что является низкими показателями, и что модель смогла уловить 30% информации о дисперсии. 
- Отсюда вывод, коэффициент детерминации R2 указывает на то, что есть еще тенденция для улучшения модели и стоит попробовать другие альтернативы.

**RandomForestRegressor**

In [10]:
# создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.01
Test MSE: 0.08
Train MAE: 0.06
Test MAE: 0.17
Train R2: 0.97
Test R2: 0.81


Стоит заметить, что метрики улучшились, но на тестовой выборке результаты хуже, чем на обучающей, что может свидетельствовать о переобучении.
- Среднеквадратическая ошибка (MSE) составила для обучающей выборки 0.1 и для тестовой выборки - 0.08. Такое различие может указывать на небольшое переобучение модели.
- Средняя абсолютная ошибка (MAE) составили 0.06 для обучающей выборки и 0.17 для тестовой выборки, что также может указывать на переобучение.
- Коэффициент детерминации (R2) составил 0.97 для обучающей выборки и 0.81 для тестовой выборки. Эти значения указывают на то, что модель довольно хорошо работает на обучающей выборке и неплохо предсказывает на тестовой выборке, хотя значение R2 на тестовой выборке заметно ниже, что также может указывать на переобучение модели. Но пока это лучшая модель.

**ElasticNetCV**

In [11]:
# создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
elastic = ElasticNetCV(cv=5, random_state=RANDOM_SEED)
elastic.fit(X_train, y_train)

# предсказания для обучающей и тестовой выборок
y_train_pred = elastic.predict(X_train)
y_test_pred = elastic.predict(X_test)

# вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.41
Test MSE: 0.40
Train MAE: 0.47
Test MAE: 0.47
Train R2: 0.06
Test R2: 0.06


Метрики ухудшились.
- Среднеквадратическая ошибка (MSE) составила для обучающей выборки 0.41 и для тестовой выборки - 0.40. Такое небольшое различие указывает, что модель не переобучалась.
- Средняя абсолютная ошибка (MAE) составили 0.47 как для обучающей выборки, так и для тестовой выборки, что означает, что модель в среднем ошибается на 0.47 в предсказаниях.
- Коэффициент детерминации (R2) составил 0.06 и для обучающей выборки и для тестовой выборки. Эти значения указывают на плохое качество модели. 

**CatBoostRegressor**

CatBoostRegressor позволяет работать с необработанными категориальными данными, в отличие от некоторых других алгоритмов машинного обучения, которые требуют предварительной обработки данных, такой как кодирование категориальных переменных в числовые значения. Для этого воспользуемся ранее сохраненным кодом.

In [12]:
# чтение данных без кодирования категориальных прихнаков
df = pd.read_csv('data/cleaned_data_for_CatBoost.csv')
df.head()

,status,propertyType,baths,city,sqft,zipcode,state,target,private_pool_final,school_rating _mean,school_dist_min,Year built,Heating_final,Cooling_encoded,Parking_encoded
0,Active,single family,-2.268674,Southern Pines,-4.111717,28387,NC,-0.584591,False,-0.478034,-2.200214,2019,True,False,False
1,For Sale,single family,-2.674134,Spokane Valley,-4.510277,99216,WA,-0.883500,False,-0.693145,-2.810405,2019,False,False,False
2,Active,single family,-3.367267,Mason City,-3.898779,50401,IA,-1.119223,False,-0.733967,-1.621481,1970,True,True,False
3,unknown,single family,-2.674134,Houston,-4.519051,77080,TX,-0.877086,False,-0.916288,-3.038531,2019,True,True,True
4,For Sale,condo,-3.367267,Flushing,-4.914413,11354,NY,-0.114288,False,-0.967581,-3.246166,1965,False,False,True


Разделим данные.

In [26]:
y = df['target']
X = df.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

In [27]:
# определим снова категориальные признаки
#cat_features = ['status', 'propertyType', 'state', 'city', 'Year built']
#param_grid = {
#    'iterations': [100, 300, 500],
#    'learning_rate': [0.01, 0.03, 0.1],
#    'depth': [4, 6, 8],
#    'l2_leaf_reg': [1, 3, 5],
#}
#cb_model = CatBoostRegressor(random_seed=RANDOM_SEED, silent=True)
#grid_search = GridSearchCV(estimator=cb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
#grid_search.fit(X_train, y_train, cat_features=cat_features)
#best_params = grid_search.best_params_
#best_cb_model = CatBoostRegressor(iterations=best_params['iterations'], learning_rate=best_params['learning_rate'], depth=best_params['depth'], l2_leaf_reg=best_params['l2_leaf_reg'], random_seed=RANDOM_SEED, silent=True)
#best_cb_model.fit(X_train, y_train, cat_features=cat_features)

#y_train_pred = best_cb_model.predict(X_train)
#y_test_pred = best_cb_model.predict(X_test)

# вычисляем метрики
#mse_train = mean_squared_error(y_train, y_train_pred)
#mse_test = mean_squared_error(y_test, y_test_pred)
#mae_train = mean_absolute_error(y_train, y_train_pred)
#mae_test = mean_absolute_error(y_test, y_test_pred) 
#r2_train = r2_score(y_train, y_train_pred)
#r2_test = r2_score(y_test, y_test_pred)

# выводим метрики
#print(f"Train MSE: {mse_train:.2f}")
#print(f"Test MSE: {mse_test:.2f}")
#print(f"Train MAE: {mae_train:.2f}")
#print(f"Test MAE: {mae_test:.2f}") 
#print(f"Train R2: {r2_train:.2f}")
#print(f"Test R2: {r2_test:.2f}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits


Так как модель очень долго обучалась, оставлю код выше с комментариями, а ниже приложу полученный результат. Рассмотрим результаты метрик.

Train MSE: 0.07

Test MSE: 0.08

Train MAE: 0.18

Test MAE: 0.19

Train R2: 0.88

Test R2: 0.85


- Среднеквадратическая ошибка (MSE) составила для обучающей выборки 0.07 и для тестовой выборки - 0.08. Такое небольшое различие может указывать на отсутсвие переобучения модели.
- Средняя абсолютная ошибка (MAE) составили 0.18 для обучающей выборки и 0.19 для тестовой выборки, что также может указывать на отсутсвие переобучения.
- Коэффициент детерминации (R2) составил 0.88 для обучающей выборки и 0.85 для тестовой выборки. Эти значения указывают на то, что модель довольно хорошо работает и в общем имеет хорошие показатели.

Вывод по итогу всех моделей обучения:
- Лучшие результаты у модели градиентного бустинга CatBoostRegressor, где показатели ошибок MSE и MAE меньшие, а коэффициент детерминации R2 наибольший для обеих как обучающей, так и тестовой выборок. И что важно, сама разница между показателями минимальна, что свидетельствует об отсутствии переобучения модели.
- RandomForestRegressor тоже имеет хороший результат обучения модели, так как он имеет наименьший показатель ошибок MSE и MAE, но при этом высокий коэффициент детерминации R2 на обеих выборках. Однако, стоит учесть переобучение случайного леса, так как разница между показателями на обучающей и тестовой наборах является существенной.